In [16]:
# Markdown cell:
# AdminDashboardDatabase.ipynb
# Purpose: admin overview, exports, simple analytics for judging.
# NOTE: This notebook includes an initial optional "drop foreign keys" cell
# that should be executed ONCE then DELETED. See instructions above.

# Code cell: imports and DB connection
import mysql.connector
import pandas as pd
import bcrypt
from datetime import datetime, timedelta

DB_CONFIG = {
    "host":"localhost",
    "user":"root",
    "password":"40418",
    "database":"aeras"
}

def get_conn():
    return mysql.connector.connect(**DB_CONFIG)

def now_bd():
    return datetime.utcnow() + timedelta(hours=6)  # Bangladesh UTC+6

print("✅ Notebook ready. Remember: run the FK-drop cell first (if needed) and then delete it.")


✅ Notebook ready. Remember: run the FK-drop cell first (if needed) and then delete it.


In [17]:
# Markdown cell: Schema check and CSV export
# This cell exports all tables to CSV for submission or backup.

conn = get_conn()
tables = ["location","points","pullers","rewards","rides","users"]
for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table};", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f"Exported {table} ({len(df)} rows)")

conn.close()


Exported location (0 rows)
Exported points (0 rows)
Exported pullers (0 rows)
Exported rewards (0 rows)
Exported rides (0 rows)
Exported users (0 rows)


C:\Users\dibya\AppData\Local\Temp\ipykernel_17076\1575663157.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table};", conn)
C:\Users\dibya\AppData\Local\Temp\ipykernel_17076\1575663157.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table};", conn)
C:\Users\dibya\AppData\Local\Temp\ipykernel_17076\1575663157.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table};", conn)
C:\Users\dibya\AppData\Local\Temp\ipyker

In [18]:
# Markdown: Admin summary & metrics
from datetime import timedelta

def admin_summary():
    conn = get_conn()
    out = {}
    out['timestamp'] = now_bd().isoformat()
    with conn.cursor(dictionary=True) as cur:
        cur.execute("SELECT COUNT(*) AS c FROM users WHERE ban=0;"); out['active_users'] = cur.fetchone()['c']
        cur.execute("SELECT COUNT(*) AS c FROM pullers WHERE ban=0;"); out['online_pullers'] = cur.fetchone()['c']
        cur.execute("SELECT COUNT(*) AS c FROM rides WHERE status IN ('REQUESTED','ACCEPTED','ACTIVE');"); out['active_rides'] = cur.fetchone()['c']
        cur.execute("SELECT COUNT(*) AS c FROM rides WHERE status='PENDING_REVIEW';"); out['pending_reviews'] = cur.fetchone()['c']

        # Top destinations
        cur.execute("SELECT dest AS destination, COUNT(*) AS cnt FROM rides GROUP BY dest ORDER BY cnt DESC LIMIT 10;")
        out['top_destinations'] = cur.fetchall()

        # Leaderboard (top by points)
        cur.execute("SELECT pullerid, points, ride_count, avg_wait_time_seconds, avg_completion_time_seconds FROM pullers ORDER BY points DESC LIMIT 10;")
        out['top_pullers'] = cur.fetchall()

    conn.close()
    return out

print(admin_summary())


{'timestamp': '2025-11-15T00:00:00.943613', 'active_users': 0, 'online_pullers': 0, 'active_rides': 0, 'pending_reviews': 0, 'top_destinations': [], 'top_pullers': []}


In [20]:
# Markdown: Additional analytics (average wait/completion, avg wait per puller)
conn = get_conn()
df_rides = pd.read_sql("SELECT rideid, request_time, accepted_time, pickup_time, drop_time, completion_time, status, userid, pullerid, wait_seconds, completion_seconds FROM rides;", conn)
conn.close()

# Convert to datetimes where necessary
df_rides['request_time'] = pd.to_datetime(df_rides['request_time'])
df_rides['accepted_time'] = pd.to_datetime(df_rides['accepted_time'])
df_rides['pickup_time'] = pd.to_datetime(df_rides['pickup_time'])
df_rides['drop_time'] = pd.to_datetime(df_rides['drop_time'])
df_rides['completion_time'] = pd.to_datetime(df_rides['completion_time'])

# Compute summary stats
summary = {
    'total_requests': len(df_rides),
    'average_wait_seconds': df_rides['wait_seconds'].dropna().mean(),
    'average_completion_seconds': df_rides['completion_seconds'].dropna().mean()
}
print(summary)


{'total_requests': 0, 'average_wait_seconds': nan, 'average_completion_seconds': nan}


C:\Users\dibya\AppData\Local\Temp\ipykernel_17076\569123048.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rides = pd.read_sql("SELECT rideid, request_time, accepted_time, pickup_time, drop_time, completion_time, status, userid, pullerid, wait_seconds, completion_seconds FROM rides;", conn)


In [15]:
schema = {}

for table in tables:
    cursor.execute(f"DESCRIBE {table};")
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=["Field", "Type", "Null", "Key", "Default", "Extra"])
    schema[table] = df

schema



{'location':                  Field          Type Null  Key Default Extra
 0             pullerid  varchar(256)  YES  MUL    None      
 1             dest_lat  decimal(9,6)  YES         None      
 2            dest_long  decimal(9,6)  YES         None      
 3        dest_distance        double  YES         None      
 4        cuet_distance        double  YES         None      
 5   pahartoli_distance        double  YES         None      
 6     naopara_distance        double  YES         None      
 7      raojan_distance        double  YES         None      
 8               status  varchar(256)  YES         None      
 9                  log      datetime  YES  MUL    None      
 10          currentlat  decimal(9,6)  YES         None      
 11         currentlong  decimal(9,6)  YES         None      
 12          created_at      datetime  YES         None      
 13          updated_at      datetime  YES         None      ,
 'points':         Field          Type Null  Key Default 